# Übungsblatt 10

**Lernziele**

In den Übungen dieser Wochen lernen Sie:

* Den Zweck von Modulen und Signaturen zu beschreiben.
* Eigene Signaturen und Module zu definieren.
* Module zu einer gegebenen Signatur zu implementieren.
* Die verschiedenen Arten zu beschreiben, auf die ein Modul eingebunden werden kann.
* Eigene Funktoren zu definieren.
* Funktoren zur Erweiterung anderer Module zu nutzen.

## Aufgabe 10.1 (P) Module und Funktoren

Was ist ein Modul und was können Module beinhalten?

In [1]:
module MeinModul =
  struct
    let a = 5
    let b = 7
    let f x = x * a
    
    type tree = Leaf | Node of int * tree * tree
    let baum = Leaf
    
    module InneresModul =
      struct
        let c = 15
        let g x = x * b
      end
  end

module MeinModul :
  sig
    val a : int
    val b : int
    val f : int -> int
    type tree = Leaf | Node of int * tree * tree
    val baum : tree
    module InneresModul : sig val c : int val g : int -> int end
  end


In [2]:
MeinModul.a

- : int = 5


In [3]:
MeinModul.f 5

- : int = 25


In [4]:
MeinModul.Leaf

- : MeinModul.tree = MeinModul.Leaf


In [5]:
MeinModul.InneresModul.c

- : int = 15


In [6]:
MeinModul.InneresModul.g 2

- : int = 14


Welchen Effekt hat die Angabe einer Signatur?

In [7]:
module type MeineSignatur =
  sig
    val a : int
    val f : int -> int
    
    type tree
    val baum : tree
    
    module InneresModul :
      sig
        val g : int -> int
      end
  end

module type MeineSignatur =
  sig
    val a : int
    val f : int -> int
    type tree
    val baum : tree
    module InneresModul : sig val g : int -> int end
  end


In [8]:
module MeinModul : MeineSignatur =
  struct
    let a = 5
    let b = 7
    let f x = x * a
    
    type tree = Leaf | Node of int * tree * tree
    let baum = Leaf
    
    module InneresModul =
      struct
        let c = 15
        let g x = x * b
      end
  end

module MeinModul : MeineSignatur


In [9]:
MeinModul.a

- : int = 5


In [10]:
MeinModul.b

error: error

In [11]:
MeinModul.Leaf

error: error

In [12]:
MeinModul.baum

- : MeinModul.tree = <abstr>


In [13]:
MeinModul.InneresModul.c

error: error

In [14]:
MeinModul.InneresModul.g 2

- : int = 14


Was ist ein Funktor? Welches Problem lässt sich damit lösen?

In [15]:
(* Bei Funktionen: *)
let f x = 2 * x
let f = fun x -> 2 * x

val f : int -> int = <fun>


val f : int -> int = <fun>


In [16]:
(* Bei Funktoren: *)
module type S = sig end
module M (X : S) = struct end
module M = functor (X : S) -> struct end

module type S = sig  end


module M : functor (X : S) -> sig  end


module M : functor (X : S) -> sig  end


Was ist der Unterschied zwischen `open` und `include`?

In [17]:
module AnderesModul =
  struct
    open MeinModul
    
    let d = a
  end

module AnderesModul : sig val d : int end


In [18]:
AnderesModul.d

- : int = 5


In [19]:
AnderesModul.a

error: error

In [20]:
module NochEinModul =
  struct
    include MeinModul
    
    let d = a
  end

module NochEinModul :
  sig
    val a : int
    val f : int -> int
    type tree = MeinModul.tree
    val baum : tree
    module InneresModul = MeinModul.InneresModul
    val d : int
  end


In [21]:
NochEinModul.d

- : int = 5


In [22]:
NochEinModul.a

- : int = 5


## Aufgabe 10.2 (P) Extending Modules

In [23]:
module List =
  struct
    include List
    
    let assoc_opt a l =
      if List.mem_assoc a l then
        Some (List.assoc a l)
      else
        None
  end

module List :
  sig
    val length : 'a list -> int
    val cons : 'a -> 'a list -> 'a list
    val hd : 'a list -> 'a
    val tl : 'a list -> 'a list
    val nth : 'a list -> int -> 'a
    val rev : 'a list -> 'a list
    val append : 'a list -> 'a list -> 'a list
    val rev_append : 'a list -> 'a list -> 'a list
    val concat : 'a list list -> 'a list
    val flatten : 'a list list -> 'a list
    val iter : ('a -> unit) -> 'a list -> unit
    val iteri : (int -> 'a -> unit) -> 'a list -> unit
    val map : ('a -> 'b) -> 'a list -> 'b list
    val mapi : (int -> 'a -> 'b) -> 'a list -> 'b list
    val rev_map : ('a -> 'b) -> 'a list -> 'b list
    val fold_left : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a
    val fold_right : ('a -> 'b -> 'b) -> 'a list -> 'b -> 'b
    val iter2 : ('a -> 'b -> unit) -> 'a list -> 'b list -> unit
    val map2 : ('a -> 'b -> 'c) -> 'a list -> 'b list -> 'c list
    val rev_map2 : ('a -> 'b -> 'c) -> 'a list -> 'b list -> 'c list
    val fold_left2 : ('a 

## Aufgabe 10.3 (P) Noch mehr Maps

Es sei die folgende Signatur gegeben, welche die grundlegende Funktionalität einer Abbildung definiert:

In [24]:
module type Map = sig
  type ('a, 'b) t
  val empty : ('a, 'b) t
  val set : 'a -> 'b -> ('a, 'b) t -> ('a, 'b) t
  val get : 'a -> ('a, 'b) t -> 'b option
  val remove : 'a -> ('a, 'b) t -> ('a, 'b) t
  val keys : ('a, 'b) t -> 'a list
end

module type Map =
  sig
    type ('a, 'b) t
    val empty : ('a, 'b) t
    val set : 'a -> 'b -> ('a, 'b) t -> ('a, 'b) t
    val get : 'a -> ('a, 'b) t -> 'b option
    val remove : 'a -> ('a, 'b) t -> ('a, 'b) t
    val keys : ('a, 'b) t -> 'a list
  end


1\. Implementieren Sie ein Modul `ListMap`, das eine Abbildung mit Hilfe einer assoziativien Liste realisiert und die Signatur `Map` erfüllt.

In [25]:
module ListMap : Map =
  struct
  
    type ('a, 'b) t = ('a * 'b) list
    
    let empty = []
    
    let remove key list = List.remove_assoc key list
    
    let set key value list = (key, value) :: remove key list
    
    let get key list =
      if List.mem_assoc key list then
        Some (List.assoc key list)
      else
        None
        
    let keys list = fst (List.split list)
  
  end

module ListMap : Map


2\. Implementieren Sie ein Modul `TreeMap`, das eine Abbildung mit Hilfe eines binären Suchbaums realisiert und die Signatur `Map` erfüllt.

In [26]:
module TreeMap : Map =
  struct

    type ('a,'b) t = Leaf | Node of 'a * 'b * ('a, 'b) t * ('a, 'b) t

    let empty = Leaf

    let rec set k v = function
      | Leaf -> Node (k, v, Leaf, Leaf)
      | Node (k', v', l, r) ->
          if k < k' then
            Node (k', v', set k v l, r)
          else if k > k' then
            Node (k', v', l, set k v r)
          else (* k = k' *)
            Node (k', v, l, r)

    let rec get k = function
      | Leaf -> None
      | Node (k', v', l, r) ->
          if k < k' then
            get k l
          else if k > k' then
            get k r
          else (* k = k' *)
            Some v'

    let rec remove_max = function
      | Leaf -> None
      | Node (k, v, l, r) ->
          match remove_max r with
            | None -> Some (k, v, l)
            | Some (max_k, max_v, new_r) ->
                Some (max_k, max_v, Node (k, v, l, new_r))

    let rec remove k = function
      | Leaf -> Leaf
      | Node (k', v', l, r) ->
          if k < k' then
            Node (k', v', remove k l, r)
          else if k > k' then
            Node (k', v', l, remove k r)
          else (* k = k' *)
            match remove_max l with
              | None -> r
              | Some (max_k, max_v, new_l) ->
                  Node (max_k, max_v, new_l, r)

    let keys m =
      let rec impl acc = function
        | Leaf -> acc
        | Node (k, _, l, r) -> impl (k :: impl acc r) l
      in
        impl [] m

  end

module TreeMap : Map


3\. Implementieren Sie einen Funktor `ExtendMap`, der ein Modul der Signatur `Map` als Argument bekommt. Der Funktor soll alle Definitionen des Arguments, sowie zusätzlich die Funktionen

```ocaml
    val pairs : ('a, 'b) t -> ('a * 'b) list
    val from_pairs : ('a * 'b) list -> ('a, 'b) t
    val values : ('a, 'b) t -> 'b list
    val map_values : ('b -> 'c) -> ('a, 'b) t -> ('a, 'c) t
    val filter_values : ('b -> bool) -> ('a, 'b) t -> ('a, 'b) t
```

zur Verfügung stellen. Definieren Sie damit anschließend ein `ExtendedListMap` und ein `ExtendedTreeMap` Modul.

In [27]:
let (%) f g x = f (g x)
let unopt = function Some x -> x | None -> failwith "unopt None"

val ( % ) : ('a -> 'b) -> ('c -> 'a) -> 'c -> 'b = <fun>


val unopt : 'a option -> 'a = <fun>


In [28]:
module ExtendMap (M : Map) =
  struct
    include M

    let pairs m = List.map (fun k -> k, M.get k m |> unopt) (M.keys m)

    let from_pairs l = List.fold_left (fun new_m (k,v) -> M.set k v new_m) M.empty l

    let values m = List.map snd (pairs m)

    let map_values f m = List.fold_left (fun new_m (k,v) -> M.set k (f v) new_m) M.empty (pairs m)

    let filter_values f m = pairs m |> List.filter (f % snd) |> from_pairs

  end

module ExtendMap :
  functor (M : Map) ->
    sig
      type ('a, 'b) t = ('a, 'b) M.t
      val empty : ('a, 'b) t
      val set : 'a -> 'b -> ('a, 'b) t -> ('a, 'b) t
      val get : 'a -> ('a, 'b) t -> 'b option
      val remove : 'a -> ('a, 'b) t -> ('a, 'b) t
      val keys : ('a, 'b) t -> 'a list
      val pairs : ('a, 'b) M.t -> ('a * 'b) list
      val from_pairs : ('a * 'b) list -> ('a, 'b) M.t
      val values : ('a, 'b) M.t -> 'b list
      val map_values : ('a -> 'b) -> ('c, 'a) M.t -> ('c, 'b) M.t
      val filter_values : ('a -> bool) -> ('b, 'a) M.t -> ('b, 'a) M.t
    end


In [29]:
module ExtendedListMap = ExtendMap (ListMap)
module ExtendedTreeMap = ExtendMap (TreeMap)

module ExtendedListMap :
  sig
    type ('a, 'b) t = ('a, 'b) ListMap.t
    val empty : ('a, 'b) t
    val set : 'a -> 'b -> ('a, 'b) t -> ('a, 'b) t
    val get : 'a -> ('a, 'b) t -> 'b option
    val remove : 'a -> ('a, 'b) t -> ('a, 'b) t
    val keys : ('a, 'b) t -> 'a list
    val pairs : ('a, 'b) ListMap.t -> ('a * 'b) list
    val from_pairs : ('a * 'b) list -> ('a, 'b) ListMap.t
    val values : ('a, 'b) ListMap.t -> 'b list
    val map_values : ('a -> 'b) -> ('c, 'a) ListMap.t -> ('c, 'b) ListMap.t
    val filter_values :
      ('a -> bool) -> ('b, 'a) ListMap.t -> ('b, 'a) ListMap.t
  end


module ExtendedTreeMap :
  sig
    type ('a, 'b) t = ('a, 'b) TreeMap.t
    val empty : ('a, 'b) t
    val set : 'a -> 'b -> ('a, 'b) t -> ('a, 'b) t
    val get : 'a -> ('a, 'b) t -> 'b option
    val remove : 'a -> ('a, 'b) t -> ('a, 'b) t
    val keys : ('a, 'b) t -> 'a list
    val pairs : ('a, 'b) TreeMap.t -> ('a * 'b) list
    val from_pairs : ('a * 'b) list -> ('a, 'b) TreeMap.t
    val values : ('a, 'b) TreeMap.t -> 'b list
    val map_values : ('a -> 'b) -> ('c, 'a) TreeMap.t -> ('c, 'b) TreeMap.t
    val filter_values :
      ('a -> bool) -> ('b, 'a) TreeMap.t -> ('b, 'a) TreeMap.t
  end
